In [198]:
# MODEL
EMBEDDING_PROVIDER="aws"
EMBEDDING_MODEL_ID="amazon.titan-embed-text-v2:0"
LLM_PROVIDER="aws"
LLM_ID = "apac.amazon.nova-lite-v1:0"
MAX_TOKEN = 1000
TEMPERATURE = 0.0
STREAMING = True

# DATABASE
REDIS_URL="redis://localhost:6379"
PERSIST_DIR="./data/chroma_dbs"
COLLECTION_NAME="parent-child-chunk-v1"
BM25_INDEX_PATH = "./data/bm25-index/bm25-index-v1.pkl"
STOPWORDS_PATH = "./data/stopwords.txt"

# RETRIEVER
RETRIEVAL_STRATEGY: str = "hybrid" 
MAX_RESULTS: int = 2
BM25_SEARCH_K: int = 5
VECTOR_SEARCH_K: int = 80
BM25_WEIGHT: float = 0.3
VECTOR_WEIGHT: float = 0.7 
RRF_CONSTANT: int = 5
SCORE_THRESHOLD: float = 0.01

# PROMPT
PROMPT_VERSION = "v1"
PROMPT_DIR = "./prompts"

# SPLITTER
PARENT_CHUNK_SIZE=2400
PARENT_CHUNK_OVERLAP=260
CHILD_CHUNK_SIZE=300
CHILD_CHUNK_OVERLAP=60

### Document Serialization

In [199]:
from typing import Any
from langchain_core.documents import Document
from langchain_core.load import dumps, loads

def encode(doc: Document) -> bytes:
    return dumps(doc).encode("utf-8")

def decode(data: bytes) -> Document:
    if isinstance(data, str):
        data = data.encode("utf-8")
    return loads(data.decode("utf-8"))

def encode_key(key: Any) -> str:
    if isinstance(key, bytes):
        return key.decode("utf-8")
    return str(key)

### Text Preprocessing

In [200]:
import os
import re
import logging
from typing import Set, List

logger = logging.getLogger(__name__)

def load_stopwords(path: str) -> Set[str]:
    try:
        if os.path.exists(path):
            with open(path, "r", encoding="utf-8") as f:
                stopwords = {line.strip().lower() for line in f if line.strip()}
            logger.info("Stopwords berhasil di load")
            return stopwords
        else:
            logger.warning(f"Stopword file tidak ditemukan {path}")
            return set()
    except Exception as e:
        logger.error(f"Gagal load stopwords: {e}")
        return set()

def preprocess_text(text: str, stopwords: Set[str]) -> List[str]:
    text = text.lower()
    tokens = re.findall(r"[a-z0-9]+(?:-[a-z0-9]+)*", text)
    if stopwords:
        tokens = [t for t in tokens if t not in stopwords]
    return tokens

### Vector Store

In [201]:
from langchain_community.vectorstores import Chroma
from langchain_core.embeddings import Embeddings

class VectorStore:
    @staticmethod
    def get_vector_store(embedding_model: Embeddings, collection_name: str = COLLECTION_NAME, persist_directory: str = PERSIST_DIR):
        return Chroma(embedding_function=embedding_model, 
                    persist_directory=persist_directory,
                    collection_name=collection_name,
                    collection_metadata={"hnsw:space": "cosine"})

### Document Store

In [202]:
import logging
from langchain_community.storage import RedisStore
from langchain.storage import EncoderBackedStore

logger = logging.getLogger(__name__)

class DocStore:
    @staticmethod
    def get_doc_store(collection_name: str = COLLECTION_NAME, redis_url: str = REDIS_URL):
        namespace = f"docstore_{collection_name}"
        try:
            logger.info(f"Connecting to Redis DocStore {redis_url}, Namespace: {namespace})")
            raw_store = RedisStore(redis_url=redis_url, namespace=namespace)
            return EncoderBackedStore(store=raw_store,
                                    key_encoder=encode_key,
                                    value_serializer=encode,
                                    value_deserializer=decode)
        except Exception as e:
            logger.error(f"Gagal koneksi ke Redis {e}")
            raise e

### Embedding

In [203]:
import os
from functools import lru_cache
from langchain_core.embeddings import Embeddings
from langchain_aws import BedrockEmbeddings

@lru_cache(maxsize=1)
def get_embeddings(provider: str = EMBEDDING_PROVIDER, model_id: str = EMBEDDING_MODEL_ID) -> Embeddings:
    if provider == "aws":
        return BedrockEmbeddings(model_id=model_id,     
                                region_name=os.getenv("AWS_REGION"))
    else:
        raise ValueError(f"Provider embedding tidak didukung")


### LLM

In [204]:
import os
from functools import lru_cache
from langchain_core.language_models import BaseChatModel
from langchain_aws import ChatBedrock

def create_llm(provider: str, model_id: str, temperature: float, max_tokens: int, streaming: bool) -> BaseChatModel:
    if provider == "aws":
        return ChatBedrock(
            model_id=model_id,
            region_name=os.getenv("AWS_REGION"),
            streaming=streaming,
            max_tokens=max_tokens,
            temperature=temperature
        )
    else:
        raise ValueError(f"Provider LLM tidak didukung")

@lru_cache(maxsize=3)
def get_llm(provider: str = LLM_PROVIDER, model_id: str = LLM_ID, temperature: float = TEMPERATURE, max_tokens: int = MAX_TOKEN, streaming: bool = STREAMING) -> BaseChatModel:
    return create_llm(provider, model_id, temperature, max_tokens, streaming)

### Parser

In [205]:
from typing import List
from langchain_core.documents import Document

def document_parser(docs: List[Document]) -> str:
    return "\n".join(doc.page_content for doc in docs)

def url_parser(docs: List[Document]) -> List[str]:
    all_urls = []
    for doc in docs:
        urls = doc.metadata.get("urls", [])
        if isinstance(urls, str):
            urls = [urls]
        all_urls.extend(urls)
    return sorted(set(all_urls))

### Prompt

In [206]:
import os
import yaml
from langchain_core.prompts import (ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate)

class Prompt:
    def __init__(self, base_dir=PROMPT_DIR):
        self.base_dir = os.path.join(os.getcwd(), base_dir, PROMPT_VERSION)
        self._cache = {}

    def get_prompt(self, name: str) -> ChatPromptTemplate:
        if name in self._cache:
            return self._cache[name]
        file_path = os.path.join(self.base_dir, f"{name}.yaml")
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Prompt not found: {file_path}")
        with open(file_path, 'r', encoding='utf-8') as f:
            data = yaml.safe_load(f)
        system_msg = SystemMessagePromptTemplate.from_template(data["system_template"])
        human_msg = HumanMessagePromptTemplate.from_template(data["human_template"])
        chat = ChatPromptTemplate.from_messages([system_msg, human_msg])
        self._cache[name] = chat
        return chat

prompt_template = Prompt()

### Retriver

In [207]:
import logging
import pickle
from typing import List, Any, Optional, Set 
from collections import defaultdict
from pydantic import PrivateAttr
from langchain_core.documents import Document
from langchain_core.runnables import RunnableSerializable, RunnableConfig

logger = logging.getLogger(__name__)

def weighted_rrf(doc_lists: List[List[Document]], weights: List[float], c: int) -> List[Document]:
    rrf_score_map = defaultdict(float)
    doc_map = {}
    for doc_list, weight in zip(doc_lists, weights):
        for rank, doc in enumerate(doc_list):
            doc_id = str(doc.metadata.get("doc_id") or hash(doc.page_content))
            if doc_id not in doc_map:
                doc_map[doc_id] = doc
            rrf_score_map[doc_id] += weight * (1 / (c + rank))
    sorted_doc_ids = sorted(rrf_score_map.keys(), key=lambda x: rrf_score_map[x], reverse=True)
    final_docs = []
    for did in sorted_doc_ids:
        doc = doc_map[did]
        doc.metadata["rrf_score"] = rrf_score_map[did]
        final_docs.append(doc)
    return final_docs

class ParentChildRetriever(RunnableSerializable):
    _vector_store: Any = PrivateAttr()
    _doc_store: Any = PrivateAttr()
    _bm25_retriever: Any = PrivateAttr(default=None)
    _stopwords: Set[str] = PrivateAttr(default_factory=set)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._vector_store = VectorStore.get_vector_store(
            embedding_model = get_embeddings(),
            collection_name = COLLECTION_NAME
        )
        self._doc_store = DocStore.get_doc_store(collection_name=COLLECTION_NAME)
        self._stopwords = load_stopwords(STOPWORDS_PATH)

    def _text_preprocessing(self, text: str):
        return preprocess_text(text, self._stopwords)

    def _load_bm25_retriever(self):
        if self._bm25_retriever:
            return self._bm25_retriever
        try:
            with open(BM25_INDEX_PATH, "rb") as f:
                retriever = pickle.load(f)
            retriever.preprocess_func = self._text_preprocessing
            retriever.k = BM25_SEARCH_K 
            self._bm25_retriever = retriever
            return retriever
        except Exception as e:
            logger.error(f"Gagal load BM25 Index: {e}")
            return None

    def _search_children_vector(self, query: str) -> List[Document]:
        results = self._vector_store.similarity_search_with_relevance_scores(
            query=query,
            k=VECTOR_SEARCH_K,
            score_threshold=SCORE_THRESHOLD,
            filter={"type": "child"}
        )
        vector_results = []
        for doc, score in results:
            if doc.metadata.get("type") == "child":
                vector_results.append(doc)
        return vector_results

    def _search_children_hybrid(self, query: str) -> List[Document]:
        # A. Vector Search
        vector_docs = self._search_children_vector(query)
        # B. BM25 Search
        bm25_retriever = self._load_bm25_retriever()
        bm25_docs = []
        if bm25_retriever:
            raw_docs = bm25_retriever.invoke(query)
            filtered_docs = [d for d in raw_docs if d.metadata.get("type") == "child"]
            bm25_docs = filtered_docs[:BM25_SEARCH_K]
        # C. RRF Scoring
        fused_docs = weighted_rrf(
            doc_lists=[vector_docs, bm25_docs],
            weights=[VECTOR_WEIGHT, BM25_WEIGHT], 
            c=RRF_CONSTANT
        )
        return fused_docs

    def _fetch_parents(self, child_docs: List[Document]) -> List[Document]:
        unique_parent_map = {}
        for child in child_docs:
            pid = child.metadata.get("parent_id")
            if pid and pid not in unique_parent_map:
                unique_parent_map[pid] = child
        parent_ids = list(unique_parent_map.keys())
        if not parent_ids:
            return []
        try:
            parent_docs = self._doc_store.mget(parent_ids)
        except Exception as e:
            logger.error(f"Redis Fetch Error: {e}")
            return []
            
        final_results = []
        for pid, p_doc in zip(parent_ids, parent_docs):
            if p_doc:
                child_ref = unique_parent_map[pid]
                rrf_score = child_ref.metadata.get("rrf_score", 0)
                p_doc.metadata["retrieval_score"] = rrf_score
                p_doc.metadata["matched_child_content"] = child_ref.page_content
                final_results.append(p_doc)
        
        final_results.sort(key=lambda x: x.metadata.get("retrieval_score", 0), reverse=True)
        return final_results[:MAX_RESULTS]

    def invoke(self, query: str, config: Optional[RunnableConfig] = None, **kwargs) -> List[Document]:
        if RETRIEVAL_STRATEGY == "hybrid":
            child_docs = self._search_children_hybrid(query)
        elif RETRIEVAL_STRATEGY == "vector":
            child_docs = self._search_children_vector(query)
        else:
            raise ValueError(f"Strategy tidak didukung")       
        if not child_docs:
            return [] 
        parent_docs = self._fetch_parents(child_docs)
        return parent_docs

In [208]:
retriever = ParentChildRetriever()
query = "Apa saja beasiswa yang tersedia?"
print(f"Query: {query}")
print("--------------")
documents = retriever.invoke(query)
for i, doc in enumerate(documents, 1):
        print(f"Dokumen {i}")
        print(f"Score : {doc.metadata.get('retrieval_score', 0)}")
        print(f"Source : {doc.metadata.get('file_path')}")
        print(f"URL : {doc.metadata.get('urls')}")
        print(f"Parent : {doc.page_content[:100]}") 
        print(f"Child : {doc.metadata.get('matched_child_content', '')[:100]}")
        print("-------------")

Query: Apa saja beasiswa yang tersedia?
--------------
Dokumen 1
Score : 0.18999999999999997
Source : MajaAI_Data\024_Jatim Cerdas.pdf
URL : ['https://majadigi.jatimprov.go.id/layanan-publik/jatim-cerdas']
Parent : 9. Double Track Dinas Pendidikan 
10. MILEA (Modernisasi Pendidikan) Dinas Pendidikan 
11. Sekolah B
Child : 9. Double Track Dinas Pendidikan 
10. MILEA (Modernisasi Pendidikan) Dinas Pendidikan 
11. Sekolah B
-------------
Dokumen 2
Score : 0.11666666666666665
Source : MajaAI_Data\036_Layanan publik kategori Pendidikan.pdf
URL : ['https://majadigi.jatimprov.go.id/layanan/layanan-magang--penelitian', 'https://majadigi.jatimprov.go.id/layanan/layanan-kerjasama-p4gn', 'https://majadigi.jatimprov.go.id/layanan/spmb-jawa-timur', 'https://majadigi.jatimprov.go.id/layanan/klinik-hoaks']
Parent : Kategori / topik
: Daftar layanan publik kategori Pendidikan 
Pertanyaan utama / intent : Apa saja l
Child : apa saja 
Jawaban : 
Daftar layanan publik di bidang pendidikan, diantaranya: 


### RAG Pipeline

In [253]:
import os 
from operator import itemgetter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

class RAGPipeline:
    def __init__(self):
        self.retriever = ParentChildRetriever()
        self.llm = get_llm(max_tokens=1000)
        self.prompt = prompt_template.get_prompt("rag")
        self.output_parser = StrOutputParser()

    def chat(self):
        retrieval = RunnableParallel({
                                    "docs": itemgetter("question") | self.retriever, 
                                    "question": itemgetter("question")
                                    })
        chat_pipeline = (
                        retrieval
                        | RunnableParallel({    
                                            "context": lambda x: document_parser(x["docs"]),
                                            "url": lambda x: url_parser(x["docs"]),
                                            "question": itemgetter("question")
                                            })
                        | self.prompt 
                        | self.llm
                        | self.output_parser
                        )
        return chat_pipeline

    def evaluate(self):
        retrieval = RunnableParallel({
            "docs": itemgetter("question") | self.retriever, 
            "question": itemgetter("question")
        })
        eval_pipeline = (
            retrieval
            | RunnableParallel({
                "context": lambda x: document_parser(x["docs"]),
                "url": lambda x: url_parser(x["docs"]),
                "question": itemgetter("question") 
            })
            | RunnableParallel({
                "answer": self.prompt | self.llm | self.output_parser,
                "context": itemgetter("context")
            })
        )
        return eval_pipeline

    def evaluate_retriever(self):
        eval_pipeline = RunnablePassthrough.assign(
            context = itemgetter("question") | self.retriever | (lambda docs: [document_parser(docs)])
        )
        return eval_pipeline

In [254]:
RAG_chat_pipeline = RAGPipeline().chat()

In [14]:
question = "Apa saja beasiswa yang tersedia?"
print(f"Pertanyaan: {question}")
for chunk in RAG_chat_pipeline.stream({"question": question}):
    print(chunk, end="", flush=True)

Pertanyaan: Apa saja beasiswa yang tersedia?
Baik, berikut uraian mengenai beasiswa yang tersedia di Provinsi Jawa Timur: 📋

1. **Beasiswa S1, S2, S3 Guru Madin Biro Kesra** - Program ini bertujuan untuk mendukung guru dalam pengembangan kompetensi akademik mereka di tingkat sarjana, magister, dan doktor.
2. **Beasiswa S1 Mahad Aly Biro Kesra** - Program ini memberikan dukungan kepada mahasiswa untuk mengejar pendidikan di tingkat sarjana di institusi tertentu.
3. **Beasiswa Al-Azhar Kairo Biro Kesra** - Program ini mendukung mahasiswa untuk mengejar pendidikan di tingkat sarjana di Universitas Al-Azhar di Kairo, Mesir.

Untuk informasi lebih lanjut, Anda dapat mengakses link-link berikut: 🌐

```
Link-Pages: [
  { title: "Jatim Cerdas", link: "https://majadigi.jatimprov.go.id/layanan-publik/jatim-cerdas" },
  { title: "Layanan Kerjasama P4GN", link: "https://majadigi.jatimprov.go.id/layanan/layanan-kerjasama-p4gn" },
  { title: "Layanan Magang & Penelitian", link: "https://majadigi.jat

## Langgraph Integration

### State

In [255]:
from typing import TypedDict, Literal
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, START, END

class ChatState(TypedDict):
    question: str
    category: Literal["info", "lapor", "ticket"]
    answer: str

### Node

In [256]:
# Router Node
class RouterOutput(BaseModel):
    category: Literal["info", "lapor", "ticket"] = Field(description="intent classification results")

llm = get_llm(max_tokens=50)
router_llm = llm.with_structured_output(RouterOutput)
router_prompt = prompt_template.get_prompt("router")
router_pipeline = router_prompt | router_llm

def router_node(state: ChatState) -> RouterOutput:
    router_output = router_pipeline.invoke({"question": state["question"]})
    state["category"] = router_output.category
    return state

def router_decision(state: ChatState) -> str:
    if state["category"] == "info":
        return "rag_node"
    elif state["category"] == "lapor":
        return "lapor_node"
    elif state["category"] == "ticket":
        return "ticket_node"
    else:
        raise ValueError("Kategori query tidak valid")

In [257]:
# RAG Node
rag = RAGPipeline()
# def rag_node(state: ChatState) -> ChatState:
#     output = rag.chat().invoke({"question": state["question"]})
#     state["answer"] = output
#     return state

async def rag_node(state: ChatState) -> ChatState:
    full_response = ""
    async for chunk in rag.chat().astream({"question": state["question"]}):
        print(chunk, end="", flush=True)
        full_response += chunk
    state["answer"] = full_response
    return state

In [258]:
# LAPOR Node
def lapor_node(state: ChatState) -> ChatState:
    state["answer"] = "LAPOR BERHASIL"
    return state

In [259]:
# TICKET Node
def ticket_node(state: ChatState) -> ChatState:
    state["answer"] = "CEK TIKET BERHASIL"
    return state

### Workflow

In [260]:
graph = StateGraph(ChatState)
graph.add_node("router_node", router_node)
graph.add_node("rag_node", rag_node)
graph.add_node("lapor_node", lapor_node) 
graph.add_node("ticket_node", ticket_node)

graph.add_edge(START, "router_node")
graph.add_conditional_edges(
    "router_node",
    router_decision,
    {"rag_node": "rag_node", 
    "lapor_node": "lapor_node", 
    "ticket_node": "ticket_node"}
)
graph.add_edge("rag_node", END)
graph.add_edge("lapor_node", END)
graph.add_edge("ticket_node", END)

workflow = graph.compile()

In [261]:
from IPython.display import display, Image
# display(Image(workflow.get_graph(xray=True).draw_mermaid_png()))
# Copy ke -> https://mermaid.live/
print(workflow.get_graph(xray=True).draw_mermaid())
# workflow.get_graph(xray=True).print_ascii()

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	router_node(router_node)
	rag_node(rag_node)
	lapor_node(lapor_node)
	ticket_node(ticket_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> router_node;
	router_node -.-> lapor_node;
	router_node -.-> rag_node;
	router_node -.-> ticket_node;
	lapor_node --> __end__;
	rag_node --> __end__;
	ticket_node --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [262]:
async def stream():
    inputs = {"question": "Apa itu KIA?"}
    async for chunk in workflow.astream(inputs, stream_mode="updates"):
        for node_name, state in chunk.items():
            if node_name == "rag_node":
                pass
            elif node_name in ["lapor_node", "ticket_node"]:
                print(state["answer"])

await stream()

Baik, berikut informasi yang dapat disampaikan mengenai KIA 😊

Kartu Identitas Anak (KIA) merupakan kartu identitas resmi untuk anak-anak Indonesia yang berusia di bawah 17 tahun dan belum menikah. KIA berfungsi layaknya KTP elektronik (e-KTP) bagi orang dewasa. KIA berwarna merah muda, berisi identitas anak tapi tanpa informasi status pekerjaan dan perkawinan. Selain itu, KIA hanya berlaku hingga anak berusia 17 tahun. Setelah berusia 17 tahun, anak diwajibkan untuk membuat e-KTP.

Sebagai identitas kependudukan resmi bagi anak-anak, KIA bisa digunakan untuk berbagai keperluan administratif, seperti pendaftaran sekolah atau beasiswa pendidikan, akses layanan kesehatan, membuka rekening bank, dan pengurusan paspor.

Syarat pendaftaran Kartu Identitas Anak baru:
1. Kartu Keluarga (KK)
2. KTP kedua orang tua
3. Akta kelahiran anak
4. Formulir pendaftaran KIA. Formulir bisa diambil di kantor Dispendukcapil atau kelurahan setempat
5. Foto berwarna ukuran 3x4 dengan background sesuai tahun 

## RAGAs Evaluation

In [753]:
import pandas as pd
import requests
import json
from tqdm.auto import tqdm
from ragas import evaluate
from ragas.metrics import LLMContextRecall, ContextPrecision
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from datasets import Dataset
from langchain_aws import BedrockEmbeddings, BedrockLLM
from langchain_aws.chat_models import ChatBedrock

In [754]:
df_eval = pd.read_csv("weekly backlog - QnA Maja.AI.csv")
total_data = len(df_eval) #60
llm_ragas = LangchainLLMWrapper(ChatBedrock(model='apac.amazon.nova-pro-v1:0', temperature=0))

C:\Users\USER\AppData\Local\Temp\ipykernel_2428\1005673746.py:3: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(ChatBedrock(model='apac.amazon.nova-pro-v1:0', temperature=0))


In [755]:
import time
RAG_eval_pipeline = RAGPipeline(prompt_version=PROMPT_VERSION).evaluate_retriever()

def get_answer_context(question:str):
    start_time = time.time()
    RAG_output = RAG_eval_pipeline.invoke({"question": question})
    context = RAG_output["context"]
    # contexts = [doc.page_content for doc in context]
    latency = time.time() - start_time
    return context, latency

### 2400 - 260 - 300 - 60 - Hybrid Search (0.3 - 0.7)

In [756]:
questions_list = []
contexts_list = []
references_list = []
latency_list = []
df_to_eval = df_eval.iloc[0:60]

for row in tqdm(df_to_eval.itertuples(), total=len(df_to_eval), desc=f"Loading {len(df_to_eval)} Data"):
    question = row.Pertanyaan
    ground_truth = row.Jawaban
    context, latency = get_answer_context(question)
    if context is not None:
        questions_list.append(question)
        contexts_list.append(context)
        references_list.append(ground_truth)
        latency_list.append(latency)
    else:
        print(f"Melewatkan pertanyaan karena error API: '{question}'")

if not questions_list:
    print("\nTidak ada data yang berhasil diproses dari API.")
else:
    data = {
        "question": questions_list,
        "contexts": contexts_list,
        "reference": references_list 
    }
    dataset = Dataset.from_dict(data)
    print(f"Evaluate RAGAs pada {len(dataset)} data")
    results = evaluate(
        dataset=dataset,
        metrics=[LLMContextRecall(), ContextPrecision()],
        llm=llm_ragas
    )
    df_result = results.to_pandas()
    latency_list_float = []
    for x in latency_list:
        try:
            latency_list_float.append(float(x))
        except:
            latency_list_float.append(None)
    df_result["latency"] = latency_list_float
    print("--- Hasil Evaluasi RAGAs ---")
    display(df_result)

Loading 60 Data: 100%|██████████| 60/60 [00:15<00:00,  3.83it/s]


Evaluate RAGAs pada 60 data


Evaluating: 100%|██████████| 120/120 [05:30<00:00,  2.75s/it]


--- Hasil Evaluasi RAGAs ---


,user_input,retrieved_contexts,reference,context_recall,context_precision,latency
0,Apa perbedaan pengurusan e-KTP online dan offl...,[Kategori\n: Dokumen Kependudukan KTP \nPertan...,Pengurusan offline dilakukan di kantor Dukcapi...,1.000000,1.0,0.313193
1,Di mana saja perekaman e-KTP dapat dilakukan s...,[Kategori\n: Dokumen Kependudukan KTP \nPertan...,Perekaman e-KTP secara offline dapat dilakukan...,1.000000,1.0,0.237265
2,Apa saja persyaratan umum dan dokumen yang har...,[Kategori\n: Dokumen Kependudukan KTP \nPertan...,"Sebelum mengurus e-KTP, pastikan Anda telah be...",1.000000,1.0,0.221443
3,Kapan seseorang wajib mengajukan penerbitan Ka...,[Kategori\n:Kartu Keluarga (KK) \nPertanyaan u...,Seseorang wajib mengajukan penerbitan Kartu Ke...,1.000000,1.0,0.219730
4,Bagaimana cara mengurus Kartu Keluarga (KK) baru?,[Kategori\n:Kartu Keluarga (KK) \nPertanyaan u...,"Untuk mengurus KK baru, pemohon perlu menyiapk...",1.000000,1.0,0.233251
5,Apa saja syarat untuk membuat KK baru karena h...,[Kategori\n:Kartu Keluarga (KK) \nPertanyaan u...,"Jika KK hilang, diperlukan surat keterangan hi...",1.000000,1.0,0.203260
6,Apa itu Kartu Identitas Anak (KIA)?,[Kategori\n: Kartu Identitas Anak (KIA) \nPert...,Kartu Identitas Anak (KIA) adalah kartu identi...,1.000000,1.0,0.235545
7,Apa syarat membuat KIA baru?,[Kategori\n: Kartu Identitas Anak (KIA) \nPert...,Syarat pembuatan KIA meliputi Kartu Keluarga (...,1.000000,1.0,0.243635
8,Di mana mengurus KIA?,[Kategori\n: Kartu Identitas Anak (KIA) \nPert...,Pengurusan KIA bisa dilakukan secara tatap muk...,1.000000,1.0,0.239507
9,Apa itu Akta Kelahiran?,[Kategori\n: Akta Kelahiran \nPertanyaan utama...,Akta Kelahiran adalah dokumen pencatatan sipil...,1.000000,1.0,0.221368


In [757]:
# With stop words
recall_mean = df_result['context_recall'].mean()
print(f"rata-rata Recall: {recall_mean}")
precision_mean = df_result['context_precision'].mean()
print(f"rata-rata Precision: {precision_mean}")
latency_mean = df_result['latency'].mean()
print(f"rata-rata Latency: {latency_mean}")

rata-rata Recall: 0.9122962962962963
rata-rata Precision: 0.9999999998999999
rata-rata Latency: 0.2596019585927328


### Cek Apakah Vector Embedding Sudah di Normalize 

In [81]:
import os
from math import sqrt
from langchain_aws import BedrockEmbeddings

MODEL_ID = "amazon.titan-embed-text-v2:0"
embedder = BedrockEmbeddings(
    model_id=MODEL_ID,
    region_name="ap-northeast-1"
)

text = "Ini adalah contoh kalimat untuk pengujian embedding."
vector = embedder.embed_query(text)
norm = sqrt(sum(v * v for v in vector))

print("Vector length:", len(vector))
print("Norm embedding:", norm)


Vector length: 1024
Norm embedding: 0.9999999983604964
